In [1]:
# VER TODO-LIST em README.md
import re, os, string, csv
import pandas as pd
import json, time, urllib.request
from reverso_context_api import Client
from tools.reverso_tools import fetch_element_as_string, get_word_data
from tools.tokenization import (get_examples, get_frequency,
                                get_text, get_tokens)
from requests.exceptions import MissingSchema
import nltk
import re

In [2]:
pape, wf = get_text("textos/pape-des-fous.txt")
TRANSLATION_NUMBER = 3
EXAMPLE_NUMBER = TRANSLATION_NUMBER
INPUT_LANGUAGE = "fr", "French"
OUTPUT_LANGUAGE = "en", "English"
client = Client(INPUT_LANGUAGE[0], OUTPUT_LANGUAGE[0])

In [3]:
# ways to perfect this function:
# check whether the first word in the concordance example is it
# if not, cut off this word (so as to remove incomplete words)
# do the same to the last word of the sentence

def boldify_selected_word(word, string):
    string_l = string.split()
    for index in range(len(string_l)):
        if string_l[index].lower() == word.lower():
            bold = "<b>"+word.lower()+"</b>"
            string_l[index] = bold 
    return string_l, bold

def get_context_and_clean_up(word, concordance):

    # get word context from concordance
    c = concordance.concordance_list((word))

    # split string and make chosen word bold
    line, bold_word = boldify_selected_word((word), c[0][6])
    word_list = line

    # remove first and last words of string, as they are sometimes incomplete
    # and assert that the first and last 
    if word_list[0] not in (word, bold_word):
        del word_list[0]
        if word_list[0] in string.punctuation:
            del word_list[0] 
    if word_list[len(word_list) - 1] not in (word, bold_word):
        del word_list[len(word_list) - 1]

    # get string with extra spaces
    unfinished_string = " ".join(word_list)
    
    # remove unnecessary spaces from string
    trailing_spaces_start = r"^\s+"
    trailing_spaces_end = r"\s+$"

    cleaned_up_pre = re.sub(r'\s([?.!,:;"](?:\s|$))', r'\1', unfinished_string)
    cleaned_up_start = re.sub(trailing_spaces_start, "", cleaned_up_pre)
    final_string = re.sub(trailing_spaces_end, "", cleaned_up_start)
    
    return final_string

def clean_up(string):
    trailing_spaces_start = r"^\s+"
    trailing_spaces_end = r"\s+$"

    cleaned_up_pre = re.sub(r'([?.!,:;"](?:\s|$))', "", string.split()[0])
    cleaned_up_start = re.sub(trailing_spaces_start, "", cleaned_up_pre)
    cleaned_up = re.sub(trailing_spaces_end, "", cleaned_up_start)

    return cleaned_up

In [4]:
tokens, token_set, phrases = get_tokens(pape)
frequency = get_frequency(tokens)
# cltk_nlp = NLP(language="grc")
# cltk_doc = cltk_nlp.analyze(songs)

wordlist = []
for pair in frequency:
    token = pair[0]
    wordlist.append(token)

In [5]:
len(wordlist)

79

In [7]:
pre_material = []
iterations = 0
for word in wordlist:
    pre_material.append((get_word_data(word, 3, 3), get_context_and_clean_up(word, wf)))
    iterations += 2
    if iterations >= 10:
        print("wait...")
        time.sleep(30)
        iterations = 0

wait...
wait...
wait...
wait...
wait...
wait...
wait...
wait...
wait...
wait...
wait...
wait...
wait...
wait...
wait...


In [21]:
for i in range(len(pre_material) - 1):
    data = pre_material[i][0]
    context = pre_material[i][1]
    md = {
        "name":fetch_element_as_string(data,"name"),
        "translations": fetch_element_as_string(data,"translations"),
        "french": fetch_element_as_string(data, "input phrases"),
        "english": fetch_element_as_string(data, "output phrases"),
        "context": context
    }
    dataframe = pd.DataFrame.from_dict(md, orient="index")
    dataframe = dataframe.transpose()
    dataframe.to_csv("french_output.csv", mode="a", header=False, index=False)
    

TypeError: 'bool' object is not subscriptable

In [6]:
get_word_data(wordlist[0], 3,3)

{'name': 'petite',
 'translations': ['small', 'little', 'smaller'],
 'examples': {'input phrases': ["J'aimerais faire une petite correction.",
   "J'aimerai vous présenter une petite...",
   'Utilisez toujours une petite quantité de pâte.'],
  'output phrases': ['I 21 would just like to make a small correction.',
   "I'd like to introduce you a small tablet...",
   'Use always a little amount of dough by time.']}}